### Bacteria model

First, we load the necessary Python modules.

In [1]:
%matplotlib widget

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as colors

Next, we define function to compute the second derivative and the overall solution numerically.

In [2]:
def discrete_laplacian(u):
    L = -2*u
    L += np.roll(u, -1)
    L += np.roll(u, +1)
    L[0]   = 2*(-u[0]+u[1])
    L[-1] = 2*(-u[-1]+u[-2])
    return L

def model_update(u, delta_t, D, r, K):
    u += (D*discrete_laplacian(u) + r*u*(1-u/K)) * delta_t
    return u

def get_initial_condition(N):
    x = np.linspace(0, 1, N)
    u = 1.5*K*(np.heaviside(x-0.4, 0) - np.heaviside(x-0.6, 0))
    return u

def update_graph(frame_id, updates_per_frame, u, delta_t, D, r, K):
    global p
    for n in range(updates_per_frame):
        u = model_update(u, delta_t, D, r, K)
    p = np.vstack((p, u))
    x = np.linspace(0, 1, N)
    u0 = get_initial_condition(N)
    line1.set_data(x, u0)
    line2.set_data(x, u)
    return [line1,line2]

In the next cell, we define the parameters of our model, set the initial condition, and run the animation of the solution.

In [3]:
# model parameters
D = 10    # diffusion coefficient
r = 2     # growth rate
K = 4     # carrying capacity

# numerical parameters
N = 200          # grid size
delta_t = 0.01   # time step

# animation parameters
N_simulation_steps = 1500
updates_per_frame = 2

# set initial condition
u = get_initial_condition(N)
p = u

# compute and animate solution
fig = plt.figure()
ax  = plt.axes(xlim=(0,1), ylim=(-0.1, np.max([K, np.max(u)])*1.1))
line1, = ax.plot([], [], linewidth=2, color = "red")
line2, = ax.plot([], [], linewidth=2, color = "blue")

u = get_initial_condition(N)
animation_arguments = (updates_per_frame, u, delta_t, D, r, K)
ani = animation.FuncAnimation(fig, update_graph, fargs=animation_arguments,
                              frames=int(N_simulation_steps/updates_per_frame),
                              interval=10, blit=True, repeat=False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Finally, we visualize the solution in a space-time contour plot.

In [5]:
fig = plt.figure()
colormapoffset = colors.TwoSlopeNorm(vmin=0, vcenter=K)
plt.imshow(p, cmap=cm.RdBu, norm=colormapoffset, extent=[0,1,0, delta_t*N_simulation_steps],
          aspect='auto', origin='lower')
plt.xlabel('Space x')
plt.ylabel('Time t')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …